# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 15 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [52]:
!pip install torchsummaryX wandb --quiet

In [53]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn.metrics
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)
import warnings
warnings.filterwarnings('ignore') # to suppress tqdm warning messages "AssertionError: can only test a child process"

Device:  cuda


In [54]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
### PHONEME LIST
PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [56]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"taeyoungchang","key":"84bd25a6cc10182ae3566a91a9d51577"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [57]:
# commands to download data from kaggle

!kaggle competitions download -c 11-785-f22-hw1p2
!mkdir '/content/data'

!unzip -qo '11-785-f22-hw1p2.zip' -d '/content/data'

11-785-f22-hw1p2.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘/content/data’: File exists


# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [62]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, context, offset=0, partition= "train", limit=-1): # Feel free to add more arguments

        self.context = context
        self.offset = offset
        self.data_path = data_path

        if partition == "train" :
          self.mfcc_dir = data_path + "train-clean-100" + "/mfcc" # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data
          self.transcript_dir = data_path + "train-clean-100" + "/transcript"  # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data
        elif partition == "valid" :
          self.mfcc_dir = data_path + "dev-clean" + "/mfcc"
          self.transcript_dir = data_path + "dev-clean" + "/transcript"  # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data

        mfcc_names = os.listdir(self.mfcc_dir) # TODO: List files in self.mfcc_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load
        transcript_names = os.listdir(self.transcript_dir) # TODO: List files in self.transcript_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load
        mfcc_names.sort()
        transcript_names.sort()

        assert len(mfcc_names) == len(transcript_names) # Making sure that we have the same no. of mfcc and transcripts

        self.mfccs, self.transcripts = [], []

        # TODO:
        # Iterate through mfccs and transcripts
        
        if limit > 0 : 
          M = limit
        else :
          M = len(mfcc_names) 

        # if limit argument is given, then load only those amount of observations.
        # else, load all observations.
        for i in range(M):
        #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir+"/"+mfcc_names[i])
        #   Optionally do Cepstral Normalization of mfcc
            mfcc = mfcc - np.mean(mfcc, axis = 1).reshape(-1,1)
        #   Load the corresponding transcript
            transcript = np.load(self.transcript_dir+"/"+transcript_names[i])[1:-1] # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
            # without traversing through the transcript?)
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

      
        # NOTE:
        # Each mfcc is of shape T1 x 15, T2 x 15, ...
        # Each transcript is of shape (T1+2) x 15, (T2+2) x 15 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that the final shape is T x 15 (Where T = T1 + T2 + ...) 

        self.mfccs = np.vstack(self.mfccs)
        # print(self.mfccs.shape)

        # TODO: Concatenate all transcripts in self.transcripts such that the final shape is (T,) meaning, each time step has one phoneme output
        
        self.transcripts = np.concatenate(self.transcripts)
        # print(self.transcripts.shape)

        assert self.mfccs.shape[0] == len(self.transcripts)
        # Hint: Use numpy to concatenate

        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, ((context , context) ,(0 , 0)), "constant", constant_values = 0 ) # TODO

        # These are the available phonemes in the transcript
        self.phonemes = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
        # But the neural network cannot predict strings as such. Instead we map these phonemes to integers

        phon_dict = {}
        for i, elem in enumerate(self.phonemes) :
          phon_dict[elem] = i
        int_map = [phon_dict.get(elem) for elem in self.transcripts]

        self.transcripts = np.array(int_map) # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.transcripts)
        # print(self.transcripts.shape)
        # print(self.mfccs.shape)  # because of zero padding, self.mfccs has 2*constant more rows than self.transcripts
        # print(self.mfccs[0:10, :])
        # print(self.transcripts[0:10])
        # print(self.mfccs[-10:, :])

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        frames = self.mfccs[ind - self.context + self.offset : ind + self.context + self.offset + 1 , :] # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames = torch.FloatTensor(frames) # Convert to tensors
        phoneme = torch.tensor(self.transcripts[ind])       

        return frames, phoneme

In [63]:
class AudioTestDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, context, offset=0, limit=-1): # Feel free to add more arguments

        self.context = context
        self.offset = offset
        self.data_path = data_path

        self.mfcc_dir = data_path + "test-clean" + "/mfcc" # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data
        
        mfcc_names = os.listdir(self.mfcc_dir) # TODO: List files in self.mfcc_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load
        
        mfcc_names.sort()

        self.mfccs = []

        # TODO:
        # Iterate through mfccs
        
        if limit > 0 : 
          M = limit
        else :
          M = len(mfcc_names) 

        # if limit argument is given, then load only those amount of observations.
        # else, load all observations.
        for i in range(M):
        #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir+"/"+mfcc_names[i])
        #   Optionally do Cepstral Normalization of mfcc
            mfcc = mfcc - np.mean(mfcc, axis = 1).reshape(-1,1)
        #   Append each mfcc to self.mfcc
            self.mfccs.append(mfcc)

      
        # NOTE:
        # Each mfcc is of shape T1 x 15, T2 x 15, ...
        # Each transcript is of shape (T1+2) x 15, (T2+2) x 15 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that the final shape is T x 15 (Where T = T1 + T2 + ...) 

        self.mfccs = np.vstack(self.mfccs)
        # print(self.mfccs.shape)
        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, ((context , context) ,(0 , 0)), "constant", constant_values = 0 ) # TODO

        # print(self.transcripts.shape)
        # print(self.mfccs.shape)  # because of zero padding, self.mfccs has 2*constant more rows than self.transcripts
        # print(self.mfccs[0:10, :])
        # print(self.transcripts[0:10])
        # print(self.mfccs[-10:, :])

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        frames = self.mfccs[ind - self.context + self.offset : ind + self.context + self.offset + 1 , :] # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames = torch.FloatTensor(frames) # Convert to tensors
    
        return frames 

    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

In [64]:
config = {
    'epochs': 100,
    'batch_size' : 1024,
    'context' : 10,
    'learning_rate' : 0.01,
    'LRscheduler' : "ReduceLROnPlateau",
    'hidden_size1' : 512,
    'hidden_size2' : 1024,
    'hidden_size3' : 1024, 
    'hidden_size4' : 1024,
    'hidden_size5' : 512,
    'dropout' : 0.1,
    'weight_decay' : 1e-5,
    'architecture' : 'trying-medium'

    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [65]:
train_data = AudioDataset(data_path = "/content/data/", partition = "train", context = config["context"], offset = config["context"]) #TODO: Create a dataset object using the AudioDataset class for the training data 

val_data = AudioDataset(data_path = "/content/data/", partition = "valid", context = config["context"], offset = config["context"]) # TODO: Create a dataset object using the AudioDataset class for the validation data 

test_data = AudioTestDataset(data_path = "/content/data/", context = config["context"], offset = config["context"]) # TODO: Create a dataset object using the AudioTestDataset class for the test data 

In [66]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True)

val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                         batch_size=config['batch_size'], pin_memory= True,
                                         shuffle= False)

test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2, 
                                          batch_size=config['batch_size'], pin_memory= True, 
                                          shuffle= False)


print("Batch size: ", config['batch_size'])
print("Context: ", config['context'])
print("Input size: ", (2*config['context']+1)*15)
print("Output symbols: ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  1024
Context:  10
Input size:  315
Output symbols:  42
Train dataset samples = 36191134, batches = 35343
Validation dataset samples = 1937496, batches = 1893
Test dataset samples = 1943253, batches = 1898


In [67]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([1024, 315]) torch.Size([1024])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [69]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, context, hidden_size1, hidden_size2, hidden_size3, hidden_size4, hidden_size5, dropout):

        super(Network, self).__init__()

        input_size = (2*context + 1) * 15 #Why is this the case?
        output_size = 40 #Why?

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, hidden_size1),
            torch.nn.BatchNorm1d(hidden_size1, track_running_stats=False),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_size1, hidden_size2),
            torch.nn.BatchNorm1d(hidden_size2, track_running_stats=False),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_size2, hidden_size3),
            torch.nn.BatchNorm1d(hidden_size3, track_running_stats=False),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_size3, hidden_size4),
            torch.nn.BatchNorm1d(hidden_size4, track_running_stats=False),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_size4, hidden_size5),
            torch.nn.BatchNorm1d(hidden_size5, track_running_stats=False),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_size5, output_size)
        )      

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [70]:
input_size = 15*(2*config['context'] + 1)
model = Network(config['context'], config["hidden_size1"], config["hidden_size2"], config["hidden_size3"], config["hidden_size4"], config["hidden_size5"], config["dropout"]).to(device)
frames,phoneme = next(iter(train_loader))
# Check number of parameters of your network - Remember, you are limited to 20 million parameters for HW1 (including ensembles)
summary(model, frames.to(device))

                         Kernel Shape  Output Shape    Params  Mult-Adds
Layer                                                                   
0_model.Linear_0           [315, 512]   [1024, 512]  161.792k    161.28k
1_model.BatchNorm1d_1           [512]   [1024, 512]    1.024k      512.0
2_model.ReLU_2                      -   [1024, 512]         -          -
3_model.Dropout_3                   -   [1024, 512]         -          -
4_model.Linear_4          [512, 1024]  [1024, 1024]  525.312k   524.288k
5_model.BatchNorm1d_5          [1024]  [1024, 1024]    2.048k     1.024k
6_model.ReLU_6                      -  [1024, 1024]         -          -
7_model.Dropout_7                   -  [1024, 1024]         -          -
8_model.Linear_8         [1024, 1024]  [1024, 1024]   1.0496M  1.048576M
9_model.BatchNorm1d_9          [1024]  [1024, 1024]    2.048k     1.024k
10_model.ReLU_10                    -  [1024, 1024]         -          -
11_model.Dropout_11                 -  [1024, 1024]

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[315, 512]","[1024, 512]",161792.0,161280.0
1_model.BatchNorm1d_1,[512],"[1024, 512]",1024.0,512.0
2_model.ReLU_2,-,"[1024, 512]",NaN,NaN
3_model.Dropout_3,-,"[1024, 512]",NaN,NaN
4_model.Linear_4,"[512, 1024]","[1024, 1024]",525312.0,524288.0
5_model.BatchNorm1d_5,[1024],"[1024, 1024]",2048.0,1024.0
6_model.ReLU_6,-,"[1024, 1024]",NaN,NaN
7_model.Dropout_7,-,"[1024, 1024]",NaN,NaN
8_model.Linear_8,"[1024, 1024]","[1024, 1024]",1049600.0,1048576.0


In [77]:
criterion = torch.nn.CrossEntropyLoss() #Defining Loss function 
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config["weight_decay"]) #Defining Optimizer
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience = 3)

# Recommended : Define Scheduler for Learning Rate, including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [78]:
torch.cuda.empty_cache()
gc.collect()

1993

In [79]:
def train(model, optimizer, criterion, dataloader):

    model.train()
    train_loss = 0.0 #Monitoring Loss
    
    for iter, (mfccs, phonemes) in enumerate(tqdm(dataloader)):

        ### Move Data to Device (Ideally GPU)
        mfccs = mfccs.to(device)
        phonemes = phonemes.to(device)

        ### Forward Propagation
        logits = model(mfccs)

        ### Loss Calculation
        loss = criterion(logits, phonemes)
        train_loss += loss.item()

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()
  
    train_loss /= len(dataloader)
    return train_loss

In [80]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode

    phone_true_list = []
    phone_pred_list = []

    valid_loss = 0.0 #Monitoring Loss

    for i, data in enumerate(tqdm(dataloader)):

        frames, phonemes = data
        ### Move data to device (ideally GPU)
        frames, phonemes = frames.to(device), phonemes.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            logits = model(frames)

            ### Loss Calculation
            loss = criterion(logits, phonemes)
            valid_loss += loss.item()

        ### Get Predictions
        predicted_phonemes = torch.argmax(logits, dim=1)
        
        ### Store Pred and True Labels
        phone_pred_list.extend(predicted_phonemes.cpu().tolist())
        phone_true_list.extend(phonemes.cpu().tolist())
        
        # Do you think we need loss.backward() and optimizer.step() here?
    
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    valid_loss /= len(dataloader)

    ### Calculate Accuracy
    accuracy = sklearn.metrics.accuracy_score(phone_pred_list, phone_true_list) 
    return valid_loss, accuracy*100

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [81]:
wandb.login(key="b604fa8b61b37bf09ec9f2b1f38b05a7b42ad93d") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [82]:
# Create your wandb run
run = wandb.init(
    name = "medium_try_first", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit=True, ### Allows reinitalizing runs when you re-run this cell
    project="hw1p2", ### Project should be created in your wandb account 
    config=config ### Wandb Config for your run
)

In [83]:
### Save your model architecture as a string with str(model) 
model_arch = str(model)

### Save it in a txt file 
arch_file = open("model_arch.txt", "w")
file_write = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20220915_175043-1v3hrh9j/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()

best_acc = 0.0 ### Monitor best accuracy in your run

for epoch in range(config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    train_loss = train(model, optimizer, criterion, train_loader)
    valid_loss, accuracy = eval(model, val_loader)
    scheduler.step(valid_loss)

    print("\tTrain Loss: {:.4f}".format(train_loss))
    print("\tValidation Accuracy: {:.2f}%".format(accuracy))


    ### Log metrics at each epoch in your run - Optionally, you can log at each batch inside train/eval functions (explore wandb documentation/wandb recitation)
    wandb.log({"train loss": train_loss, "validation accuracy": accuracy})

    ### Save checkpoint if accuracy is better than your current best
    if accuracy >= best_acc:

      ### Save checkpoint with information you want
      torch.save({'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': train_loss,
              'acc': accuracy}, 
        './model_checkpoint.pth')
      
      ### Save checkpoint in wandb
      wandb.save('checkpoint.pth')

    # Is your training time very high? Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
    # Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

### Finish your wandb run
run.finish()


Epoch 1/100


  0%|          | 0/35343 [00:05<?, ?it/s]

  0%|          | 0/1893 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f1a409ff440>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f1a409ff440>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
        self._shutdown_workers()self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers

      File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'if w.is_alive():
AssertionError: 
can only test a child process  File "/usr/lib/python

	Train Loss: 1.0577
	Validation Accuracy: 69.33%

Epoch 2/100


...

  0%|          | 0/1893 [00:00<?, ?it/s]

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f1a409ff440>
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f1a409ff440>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
        self._shutdown_workers()self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers

      File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
if w.is_alive():
      File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
if w.is_alive():    
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
assert self._parent_pid == os.getpid(), 'can only test a child

	Train Loss: 0.8114
	Validation Accuracy: 75.42%

Epoch 16/100


  0%|          | 0/35343 [00:05<?, ?it/s]

Exception ignored in: Exception ignored in: Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f1a409ff440><function _MultiProcessingDataLoaderIter.__del__ at 0x7f1a409ff440><function _MultiProcessingDataLoaderIter.__del__ at 0x7f1a409ff440>
Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f1a409ff440>

  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__

Traceback (most recent call last):
Traceback (most recent call last):
    Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
self._shutdown_workers()  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__

      File "/usr/local/lib/python3.7/dist-packages/torch/utils/

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *random_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
  ### What you call for model to perform inference?
  model.eval()

  ### List to store predicted phonemes of test data
  test_predictions = []

  ### Which mode do you need to avoid gradients?
  with torch.inference_mode() :

      for i, frames in enumerate(tqdm(test_loader)):

          frames = frames.float().to(device)             
          
          output = model(frames)

          ### Get most likely predicted phoneme with argmax
          predicted_phonemes = torch.argmax(output, dim=1)

          ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
          test_predictions.extend(predicted_phonemes.cpu().tolist())

  return test_predictions

In [ ]:
predictions = test(model, test_loader)

  0%|          | 0/1898 [00:00<?, ?it/s]

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API
!kaggle competitions submit -c 11-785-f22-hw1p2 -f ./submission.csv -m "Test Submission"

100% 18.5M/18.5M [00:04<00:00, 4.43MB/s]
Successfully submitted to Frame-Level Speech Recognition

In [ ]:
val_data = AudioDataset(data_path = "/content/data/", partition = "valid", context = 0, offset = 0)

In [ ]:
val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                         batch_size=200, pin_memory= True,
                                         shuffle= False)

In [ ]:
# Testing code to check if your data loaders are working
sum = 0
for i, data in enumerate(val_loader):
    frames, phoneme = data
    sum += (phoneme == 0).nonzero().shape[0]
print(sum)

396367


In [ ]:
arr = phoneme.cpu().detach().numpy()

In [ ]:
arr

array([35, 35, 21, 21, 21, 21, 21, 21, 21, 17, 17, 17, 17,  7,  7,  7,  7,
        7,  7,  7, 12, 12, 12, 12, 12, 31, 31, 31, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [ ]:
values, counts = np.unique(arr, return_counts=True)

In [ ]:
values

array([ 0,  7, 12, 17, 18, 21, 31, 35])

In [ ]:
counts

array([49,  7,  5,  4, 19,  7,  3,  2])

In [ ]:
count_dict = {}
for i in range(40) :
  count_dict[i] = 0

In [ ]:
# Testing code to check if your data loaders are working
counts_array = np.zeros(40, dtype = "int")
for i, data in enumerate(val_loader):
    frames, phoneme = data
    array = phoneme.cpu().detach().numpy()
    values, counts = np.unique(array, return_counts = True)
    for i, elem in enumerate(values) :
      counts_array[elem] += counts[i]
print(counts_array)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe27f8ba710>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe27f8ba710>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

[396367  33196  55212  88799  27736  21036  47481  22436  14144  48215
  28445  42820  56102  38939  35969  13627  26904  63752  81199   9396
  46601  73571  43783  85424  21730  31452   3752  31371  59326 111876
  20220  84166  11322   5713  20977  24287  37897  11732  59360   1161]


In [ ]:
np.argsort(counts_array)

array([39, 26, 33, 19, 32, 37, 15,  8, 30, 34,  5, 24,  7, 35, 16,  4, 10,
       27, 25,  1, 14, 36, 13, 11, 22, 20,  6,  9,  2, 12, 28, 38, 17, 21,
       18, 31, 23,  3, 29,  0])

In [ ]:
PHONEMES[39]

'ZH'